In [1]:
%load_ext jupyter_black

import os
from datetime import datetime
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import glob
from shapely.geometry import Point
from datetime import timedelta
import warnings


In [2]:
warnings.filterwarnings("ignore")
AA_DATA_DIR = os.getenv("AA_DATA_DIR")
save_dir = Path(AA_DATA_DIR) / "public" / "exploration" / "moz" / "ecmwf_hindcast"
ADMS = ["Sofala", "Inhambane", "Nampula", "Zambezia"]
adm1_path = (
    Path(AA_DATA_DIR)
    / "public"
    / "raw"
    / "moz"
    / "cod_ab"
    / "moz_admbnda_adm1_ine_20190607.shp"
)
gdf_adm1 = gpd.read_file(adm1_path)
gdf_sel = gdf_adm1[gdf_adm1.ADM1_PT.isin(ADMS)]

In [14]:
def categorize_cyclone(wind_speed):
    if wind_speed > 115:
        return "Very Intense Tropical Cyclone"
    elif wind_speed >= 90:
        return "Intense Tropical Cyclone"
    elif wind_speed >= 64:
        return "Tropical Cyclone"
    elif wind_speed >= 48:
        return "Severe Tropical Storm"
    elif wind_speed >= 34:
        return "Moderate Tropical Storm"
    elif wind_speed >= 28:
        return "Tropical Depression"
    else:
        return "Tropical Disturbance"

In [46]:
moz_cyclones = [
    Path(f).stem.replace("_all", "").upper()
    for f in glob.glob(str(Path(save_dir) / "csv" / "*_all.csv"))
]
cyclone_speed = []
for cyclone_file_path in glob.glob(str(save_dir / "csv/*_all.csv")):
    cyclone_name = Path(cyclone_file_path).stem.split("_")[0]
    print(f"Processing file: {cyclone_file_path}")
    cyclone_file = pd.read_csv(cyclone_file_path)
    cyclone_file["time"] = pd.to_datetime(cyclone_file["time"])

    cyclone_df = (
        cyclone_file[["time", "speed", "lat", "lon", "lead_time", "forecast_time"]]
        .groupby(["time", "forecast_time"])
        .median()
        .reset_index()
    )
    cyclone_df["lat"] = cyclone_df["lat"].apply(lambda x: -x if x > 0 else x)

    cyclone_df["speed_knots"] = cyclone_df["speed"] * 1.94384
    cyclone_df["storm_category"] = cyclone_df["speed_knots"].apply(categorize_cyclone)
    lt_0_df = cyclone_df[cyclone_df["lead_time"] == 0]
    lt_0_df = gpd.GeoDataFrame(
        lt_0_df, geometry=gpd.points_from_xy(lt_0_df.lon, lt_0_df.lat), crs="EPSG:4326"
    )
    cyc_sjoin = gpd.sjoin(lt_0_df, gdf_sel, how="left", predicate="intersects")
    lt_0_df["within_land"] = cyc_sjoin["index_right"].notna()
    lt_0_df["point_location"] = np.where(lt_0_df["within_land"], "Within", "Outside")
    if all(~lt_0_df["within_land"]):
        continue
    first_landfall = (
        lt_0_df[lt_0_df["within_land"]].index[0]
        if not lt_0_df[lt_0_df["within_land"]].empty
        else None
    )
    lt_0_df.loc[lt_0_df.index == first_landfall, "point_location"] = "Landfall"
    landfall_time = pd.to_datetime(
        lt_0_df[lt_0_df["point_location"] == "Landfall"]["time"].values[0]
    )
    cyc_df = lt_0_df[lt_0_df["point_location"] != "Outside"]
    cyc_df["storm"] = cyclone_name.upper()
    cyclone_speed.append(cyc_df)

pd.concat(cyclone_speed)

Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\favio_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\jokwe_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\izilda_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\dando_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\irina_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\haruna_all.

,time,forecast_time,speed,lat,lon,lead_time,speed_knots,storm_category,geometry,within_land,point_location,storm
204,2007-02-22 12:00:00,"2007/02/22, 12:00:00",14.30,-21.3,35.0,0.0,27.796912,Tropical Disturbance,POINT (35 -21.3),True,Landfall,FAVIO
225,2007-02-23 00:00:00,"2007/02/23, 00:00:00",15.50,-19.7,34.1,0.0,30.129520,Tropical Depression,POINT (34.1 -19.7),True,Within,FAVIO
48,2008-03-08 12:00:00,"2008/03/08, 12:00:00",17.70,-15.5,40.3,0.0,34.405968,Moderate Tropical Storm,POINT (40.3 -15.5),True,Landfall,JOKWE
63,2008-03-09 00:00:00,"2008/03/09, 00:00:00",18.00,-16.6,39.3,0.0,34.989120,Moderate Tropical Storm,POINT (39.3 -16.6),True,Within,JOKWE
3,2012-01-16 12:00:00,"2012/01/16, 12:00:00",20.50,-24.4,34.3,0.0,39.848720,Moderate Tropical Storm,POINT (34.3 -24.4),True,Landfall,DANDO
120,2019-03-15 00:00:00,"2019/03/15, 00:00:00",35.00,-19.6,34.5,0.0,68.034400,Tropical Cyclone,POINT (34.5 -19.6),True,Landfall,IDAI
3,2022-01-24 12:00:00,"2022/01/24, 12:00:00",21.10,-16.2,38.9,0.0,41.015024,Moderate Tropical Storm,POINT (38.9 -16.2),True,Landfall,ANA
8,2022-01-25 00:00:00,"2022/01/25, 00:00:00",17.50,-16.0,36.0,0.0,34.017200,Moderate Tropical Storm,POINT (36 -16),True,Within,ANA
63,2022-03-11 12:00:00,"2022/03/11, 12:00:00",14.65,-15.1,39.3,0.0,28.477256,Tropical Depression,POINT (39.3 -15.1),True,Landfall,GOMBE
80,2022-03-12 00:00:00,"2022/03/12, 00:00:00",10.80,-14.5,37.6,0.0,20.993472,Tropical Disturbance,POINT (37.6 -14.5),True,Within,GOMBE
